In [1]:
import json
import csv
from tqdm import tqdm_notebook as tqdm

In [2]:
words = []
dataset = []
with open("../datasets/dataset.csv", "r") as f:
    tags = csv.reader(f)
    header = next(tags)
    
    for t in tags:
        words += t
        dataset += [t]

In [3]:
len(set(words))

66

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import time

In [6]:
# Qiitaからタグ検索
def tagSearch(w, page="1"):
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome(chrome_options=options)
    driver.get('https://teratail.com/tags/' + w + '/active/' + page)
    data = driver.page_source
    html = BeautifulSoup(data, 'lxml')
    driver.close()
    driver.quit()
    return html

In [7]:
res_d = dict()

for w in tqdm(set(words)):
    res = []
    for p in range(1, 3):
        tag_list = tagSearch(w, page=str(p)).find_all(class_="entry-tags boxTag j-questionTagsInQuestionsList p-questionFeedTagList")
        if tag_list:
            for i in tag_list:
                res += [[j.text.lower() for j in i.find_all("a")]]
                time.sleep(0.5)
        else:
            break
    
    res_d[w] = res

In [9]:
res_d.keys()

dict_keys(['phina.js', 'node.js', 'yarn', 'rbenv', 'ecmascript', 'vuetify', 'cakephp', 'bootstrap', 'nuxt.js', 'vuex', 'rstudio', 'beautifulsoup', 'r', 'scss', 'scikit-learn', 'jquery', 'ruby', 'kivy', 'rspec', 'promise', 'javascript', 'keras', 'css', 'three.js', 'webpack', 'mruby', 'd3.js', 'react', 'ajax', 'tkinter', 'symfony', 'spring', 'html', 'babel', 'pytorch', 'sass', 'reactnative', 'pyqt', 'next.js', 'eslint', 'pip', 'coffeescript', 'pyenv', 'capybara', 'jupyter', 'java', 'php', 'python', 'backbone.js', 'scipy', 'numpy', 'vue.js', 'canvas', 'typescript', 'npm', 'chrome-extension', 'chainer', 'flask', 'rails', 'angular', 'electron', 'javafx', 'webgl', 'django', 'redux', 'laravel'])

In [10]:
with open('terateil_results_data.json', 'w') as f:
    json.dump(res_d, f)

In [11]:
with open('terateil_results_data.json', 'r') as f:
    read_res_d = json.load(f)

In [26]:
read_res_d["react"]

[]

## 存在しないタグの数と取得できたタグの数

In [37]:
c = 0
cl = []
for r in read_res_d:
    #print(len(read_res_d[r]))
    if not read_res_d[r]:
        c += 1
        cl.append(r)
        
print(c)
print(cl)

13
['phina.js', 'vuetify', 'scss', 'beautifulsoup', 'rstudio', 'promise', 'mruby', 'react', 'rails', 'chrome-extension', 'reactnative', 'pyenv', 'scipy']


In [21]:
# バージョンを表すテキストを削除
def eraseVersion(w):
    w = re.sub('(\d+(?:\.\d+)?)$', '', w)
    return w

In [15]:
# w2に対するw1のスコアを返す
def check(w1, w2):
    w1 = w1.lower()
    w2 = w2.lower()
    res = []
    score = 0
    count = 0
        
    # w1で検索した際にw2が含まれていたら1 それ以外は0
    for i in read_res_d[w1]:
        res += [[eraseVersion(j.lower())==w2 for j in i].count(True)]
        count += 1
    time.sleep(0.1)

    # 検索結果が存在しない場合スコア0を返す
    if len(res) == 0:
        return 0

    # w1で検索した時にw2が含まれる割合
    score += ((len(res)-res.count(0))+1e-7)/(len(res)+1e-7)

    # 1つのページの出現率が30%以下ならスコア0を返す
    #if score/p < 0.2:
    #    return 0
    
    #print(len(res)-res.count(0))
        
    return round(score, 3)

In [32]:
# 2つの単語を比較
def comparisonCheck(w1, w2, boundary=0.2, p=True):
    w1_score = check(w1, w2)
    w1, w2 = w2, w1
    w2_score = check(w1, w2)
    w1, w2 = w2, w1
    
    if p:
        print(w1, "Dscore:", w1_score)
        print(w2, "Dscore:", w2_score)
        
    if (w1_score < boundary and w2_score < boundary) or (abs(w1_score-w2_score) < 0.1):
        if p:
            print(w1, "and", w2, "are irrelevant")
        return {}
    else:
        if w1_score < w2_score:
            if p:
                print(">>", w1, "is easy!")
            return {"easy": w1, "hard": w2, "score": max(w1_score, w2_score)}
        else:
            if p:
                print(">>", w2, "is easy!")
            return {"easy": w2, "hard": w1, "score": max(w1_score, w2_score)}

In [33]:
comparisonCheck("javascript", "html")

javascript Dscore: 0.375
html Dscore: 0.425
javascript and html are irrelevant


{}

In [19]:
len(dataset)

60

In [36]:
correct = 0
check_res = []

for ds in tqdm(dataset):
    
    cr = comparisonCheck(ds[0], ds[1], p=False)
    if cr:
        if [cr["easy"], cr["hard"]] == [ds[0], ds[1]]:
            correct += 1
        else:
            print([ds[0], ds[1]])
    else:
        print([ds[0], ds[1]])
            
print(correct/(len(dataset)))

['javascript', 'react']
['javascript', 'promise']
['javascript', 'chrome-extension']
['javascript', 'phina.js']
['javascript', 'npm']
['html', 'electron']
['css', 'scss']
['vue.js', 'vuetify']
['webgl', 'three.js']
['react', 'redux']
['react', 'next.js']
['react', 'reactnative']
['ruby', 'rails']
['ruby', 'mruby']
['python', 'pyenv']
['python', 'scipy']
['python', 'beautifulsoup']
['r', 'rstudio']

0.7


## 存在しないデータを除いた場合

In [44]:
correct = 0
count = 0
check_res = []

for ds in tqdm(dataset):
    
    if not (ds[0] in cl or ds[1] in cl):
        cr = comparisonCheck(ds[0], ds[1], p=False)
        if cr:
            if [cr["easy"], cr["hard"]] == [ds[0], ds[1]]:
                correct += 1
            else:
                print([ds[0], ds[1]])
        else:
            print([ds[0], ds[1]])
        count += 1

print(correct/count)

['javascript', 'npm']
['html', 'electron']
['webgl', 'three.js']

0.9333333333333333


In [46]:
correct, count

(42, 45)